# Extracting Text From Websites

Now that I have a list of websites that may contain information related to Biblical archeology, I need to extract the actual text from those articles in order to analyze each article and decide it it is infact related to Biblical archeology. I will later use this data to create a model that predicts the articles that I would arctually be interested in. I will also use this data to identify any references to specific Bible verses within an article. 

This site describes how to extract only the text data from a website: https://www.dataquest.io/blog/web-scraping-tutorial-python/

In [13]:
import re
import requests
from bs4 import BeautifulSoup
from pandas_ods_reader import read_ods
import pandas as pd
import sqlite3
import time

I am going to develop this concept using the following website as my subject: https://en.wikipedia.org/wiki/Cana#Written_references_to_Cana

In [14]:
url = 'https://en.wikipedia.org/wiki/Cana#Written_references_to_Cana'

Next, I will use *requests.get* to open this website and see if there are any errors.

In [15]:
website = requests.get(url)
website

<Response [200]>

It looks like everything worked, so next I will use *BeautifulSoup*.

In [16]:
soup = BeautifulSoup(website.content, 'html.parser')

In [18]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Cana - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"fbd0444a-95e8-48f4-9f1f-9d2789646944","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Cana","wgTitle":"Cana","wgCurRevisionId":969504170,"wgRevisionId":969504170,"wgArticleId":432485,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["All articles with dead external links","Articles with dead external links from December 2017","Articles with permanently dead external links","CS1 uses Hebrew-language script (he)

That was pretty easy, but clearly, I extracted way more information than I need. I will filter to just the interesting text using the tag "p".

I will also remove some unwanted characters at this point.

In [27]:
text = ' '.join([item.get_text() for item in soup.find_all('p')]).replace('\n','').replace('\xa0','')

In [28]:
text

'Cana of Galilee (Ancient Greek: Κανὰ τῆς Γαλιλαίας, Arabic: قانا الجليل\u200e, romanized:Qana al-Jalil, lit.\'Qana of the Galilee\') is the location of the Marriage at Cana, at which the miracle of turning water into wine took place in the Gospel of John.  The location is disputed, with the four primary locations being  Kafr Kanna, Khirbet Qana and Reineh in Lower Galilee and Qana in Upper Galilee. The Arabic name "Qana al-Jalil" has been said to apply to a number of sites, but is of doubtful authenticity.[1]The name possibly derives from the Hebrew or Aramaic word for reeds.[2] Among Christians and other students of the New Testament, Cana is best known as the place where, according to the Fourth Gospel, Jesus performed "the first of his signs", his first public miracle, the turning of a large quantity of water into wine at a wedding feast (John 2:1–11) when the wine provided by the bridegroom had run out. Although none of the synoptic gospels record the event, mainstream Christian t

This looks great! We still have a ton of processing to do before we can use this text to make predictions, but this is exactly the text I want to use.

# Creating a Dataset of Extracting Text
Now that I have extract text from a single website, I want to create a dataset that contains all of the extracted text from the websites I have labeled.

First, I have to import the ODS file that contains the labeled websites.

In [1]:
import re
import requests
from bs4 import BeautifulSoup
from pandas_ods_reader import read_ods
import pandas as pd
import sqlite3
import time

In [2]:
path = "C:/Bible Research/data/labeled websites - no gov.ods"

sheet_idx = 1
df = read_ods(path, sheet_idx)

In [3]:
len(df)

2527

In [4]:
df.relevant.sum()

490.0

This shows that I read and labeled 2,605 articles about Biblical archeology. These are the articles that my web crawler pulled from Google in relation to Biblical archeology and the nations mentioned in the Bible. Of these, I felt like 491 were relevant to the topic of Biblical archeology. I felt like the others were tangental, i.e. sermon notes that mention Biblical archeology but added nothing to the topic. 

Note: Labeling this data is a likely source of noise. While I tried very hard to be consistent in my labeling, this process took hundreds of hours in an effort that lasted nearly half a year. Since this effort was stretched out across such a long time period, there is every reason to believe that I may have inconsistently labeled sites. In an attempt to control for this, once I read and labeled the entire list, I reread all of the relevant articles and decided again whether or not they were truly relevant. Unfortunately, I did not have time to reread all of the non-relevant articles to form a second opinion. I'm somewhat interested in seeing which non-relevant articles my predictive model will classify as relevant and how relevant they actually are. The worst possible outcome would be if there is so much noise that my model cannot accurately classify relevant articles. If it inaccurately classifies non-relevant articles, I can always read those and make an informed decision. If it inaccurately classifies relevant articles, these are simply lost.

Before I extract the text data, I need to create an empy dataset in which to place the extacted text. I want to keep the website and whether or not it's relevant. I also want to create a counter variable. Also, if the function *visible* has not been run, than should be done before proceeding.

In [5]:
My_Text = pd.DataFrame(columns=['website', 'relevant', 'text'])
n = 0

Next, I'm going to use a FOR loop to iterate through these websites and extract the text for each. Because this FOR loop is unexpectedly stopping and I'm loosing all of the great text that's been extracted, I'm going to do this in small sections. This will take more time, but I feel like it's the best way to get the information.

In [6]:
for index, row in df[2240:].iterrows():
    
    print(row[0])
        
    try:
        website = requests.get(row[0])
    except:
        continue
    
    soup = BeautifulSoup(website.content, 'html.parser')
        
    j = ' '.join([item.get_text() for item in soup.find_all('p')]).replace('\n','').replace('\xa0 ','')
    
    My_Text.loc[n] = [row.website] + [row.relevant] + [j]
    n+=1
    
    time.sleep(2)
    print(n)

https://www.nytimes.com/1982/04/25/us/israeli-archeologists-end-12-years-of-work-in-sinai.html
1
https://www.nytimes.com/1990/02/22/world/believers-score-in-battle-over-the-battle-of-jericho.html
2
https://www.nytimes.com/1996/07/23/science/inscription-at-philistine-city-shows-this-is-the-right-place.html
3
https://www.nytimes.com/1997/01/19/us/james-pritchard-87-a-biblical-archeologist.html
4
https://www.nytimes.com/1998/01/24/books/think-tank-did-amateur-gold-diggers-find-mount-sinai-the-pros-doubt-it.html
5
https://www.nytimes.com/2002/10/21/science/artifact-may-be-earliest-relating-to-existence-of-jesus.html
6
https://www.nytimes.com/2018/01/02/nyregion/lawrence-stager-creative-biblical-archaeologist-dies-at-74.html
7
https://www.nytimes.com/2018/11/30/world/middleeast/pontius-pilate-ring.html
8
https://www.nytimes.com/2019/07/03/science/philistines-dna-origins.html
9
https://www.olivetree.com/blog/archaeological-study-bible/
10
https://www.parks.org.il/en/reserve-park/mount-tabor-

In [7]:
len(My_Text)

283

Not bad! I extracted text from 447 of the 450 sites I used. I will rename this dataframe and run the next batch. I'll do this until I've iterated through the entire list of websites.

In [8]:
conn = sqlite3.connect(r"C:\Bible Research\SQL database\biblesql.db")

In [9]:
My_Text_Full = pd.read_sql('select * from labeled_text2', conn)

In [10]:
c = pd.concat([My_Text_Full,My_Text],ignore_index=True)

In [11]:
len(c)

2507

In [12]:
c.to_sql('labeled_text2', conn, if_exists='replace', index=False)

In [13]:
pd.read_sql('select count(*) as Count from labeled_text2', conn)

,Count
0,2507


In [16]:
pd.read_sql('select count(*) as Count from labeled_text', conn)

,Count
0,1991


In [14]:
pd.read_sql('select * from labeled_text2 limit 1', conn)

,website,relevant,text
0,http://apologeticspress.org/apcontent.aspx?cat...,1.0,"Almost fifty times in the Old Testament, we..."


[('bible_bbe',), ('book_key',), ('books',), ('bible_metrics',), ('labeled_text',), ('labeled_text2',)]


Using *requests*, I was able to access 2,507 of the 2,527 labeled websites. Using *urllib*, I was only able to access 1,991. At first glance, it appears that *requests* did a better job, but I want to see if I actually got data from the sites I acccessed with either, since that will determine which did better.

In [20]:
pd.read_sql('select count(*) as Text_Count from labeled_text2 where length(text) > 0', conn)

,count(*)
0,2071


Since there are over 1,991 sites for which we have text data using *request*, I already know that this method did a better job. However, I'm interested in knowing how much better it did.

In [22]:
pd.read_sql('select count(*) as Text_Count from labeled_text where length(text) > 0', conn)

,Text_Count
0,1788


WOW! Much better. *requests* provides text data for nearly 300 more sites than *urllib*. That's over 15% more data.

Next, it's possible that *urllib* provides text data for some sites that *requests* does not. I will filter to the websites that have text data for both methods and query the results to see if there is any additional data.

In [126]:
sites2 = pd.read_sql('select * from labeled_text2 where length(text) > 0', conn)

In [127]:
filter = list(sites.website)

In [129]:
sites1 = pd.read_sql('select * from labeled_text where length(text) > 0', conn)

In [130]:
sites1.head()

,website,relevant,text
0,http://apologeticspress.org/apcontent.aspx?cat...,1.0,"Almost fifty times in the Old Testament, we ca..."
1,http://apologeticspress.org/article/1217,1.0,The biblical accounts of the travels of Paul o...
2,http://apologeticspress.org/article/1347,1.0,A man wearing a leather vest and a broad-rimme...
3,http://apologeticspress.org/article/852,1.0,A favorite argument against the Bible’s inspir...
4,http://bible7evidence.blogspot.com/2014/09/abr...,1.0,Hello friends! The following is evidence of Ab...


In [131]:
additions = sites1[~sites1['website'].isin(filter)]

It looks like there is text data for 79 additional sites using the first method. I will incorportate these into the final dataset.

In [133]:
sites2.head()

,website,relevant,text
0,http://apologeticspress.org/apcontent.aspx?cat...,1.0,"Almost fifty times in the Old Testament, we..."
1,http://apologeticspress.org/article/1217,1.0,\r\tThe biblical accounts of the travels of...
2,http://apologeticspress.org/article/1347,1.0,A man wearing a leather vest and a broad-ri...
3,http://apologeticspress.org/article/852,1.0,\r\tA favorite argument against the Bible’s...
4,http://bible7evidence.blogspot.com/2014/09/abr...,1.0,Post a Comment


In [134]:
additions.head()

,website,relevant,text
8,http://ed5015.tripod.com/BCush124.html,1.0,yet there’s no proof of Cushite control over E...
16,http://oldtestamentstudies.datascenesdev.com/c...,1.0,"This page summarises the history of Gibeah, a..."
18,http://thehiddenmission.com/forum/showthread.p...,1.0,Quote: The stone block found in the sanctuary ...
21,http://www.biblehistory.net/newsletter/baalis.htm,1.0,"After Lot fled the city of Sodom, which God de..."
22,http://www.biblehistory.net/newsletter/balaam.htm,1.0,"In the country of Jordan, at an excavation sit..."


In [136]:
final = sites2.append(additions,ignore_index=True)

In [137]:
len(final.index)

2150

By combining methods, I was able to extract data for 85% of the labeled websites.

In [142]:
final.to_sql('labeled_text', conn, if_exists='replace', index=False)

In [147]:
pd.read_sql('select count(*) as Count from labeled_text', conn)

,Count
0,2150


In [156]:
pd.read_sql('select count(*) from labeled_text2 where length(text) > 500', conn)

,count(*)
0,1838


In [157]:
pd.read_sql('select count(*) from labeled_text2 where length(text) < 500', conn)

,count(*)
0,669


In [162]:
pd.read_sql('select count(*) from labeled_text2 where length(text) > 500', conn)

In [163]:
reduced.to_sql('reduced_text', conn, if_exists='replace', index=False)

In [164]:
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('bible_bbe',), ('book_key',), ('books',), ('bible_metrics',), ('labeled_text2',), ('labeled_text',), ('reduced_text',)]


In [ ]:
#cursor.execute('Drop table set1')